In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Installing Requirements**

In [49]:
!pip install gradio==3.50


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 62.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.15.0
    Uninstalling gradio_client-0.15.0:
      Successfully uninstalled gradio_client-0.15.0
  Attempting uninstall: gradio
    Found existing installation: gradio 4.25.0
    Uninstalling gradio-4.25.0:
      Successfully uninstalled gradio-4.25.0


In [2]:
!conda install -y -c conda-forge sox

Retrieving notices: ...working... done
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - sox


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lame-3.100                 |    h166bdaf_1003         496 KB  conda-forge
    libflac-1.4.3              |       h59595ed_0         385 KB  conda-forge
    libogg-1.3.4               |       h7f98852_1         206 KB  conda-forge
    libopus-1.3.1              |       h7f98852_1         255 KB  conda-forge
    libsndfile-1.2.2           |       hc60ed4a_1         346 KB  conda-forge
    libvorbi

**Clone the repository**

In [8]:
# Clone the repository
!git clone https://github.com/m23csa007/LibriMix
    
# move to LibriMix directory
%cd LibriMix

# Create the dataset
!bash generate_librimix.sh /kaggle/working/storage_dir

Cloning into 'LibriMix'...
remote: Enumerating objects: 308, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 308 (delta 89), reused 58 (delta 58), pack-reused 163
Receiving objects: 100% (308/308), 33.44 MiB | 21.28 MiB/s, done.
Resolving deltas: 100% (134/134), done.
/kaggle/working/LibriMix
Creating mixtures and sources from metadata/myLibri2Mix/libri2mix_test-clean.csv in /kaggle/working/Libri2Mix/wav8k/min/test
100%|███████████████████████████████████████| 3000/3000 [00:48<00:00, 62.08it/s]
Creating mixtures and sources from metadata/myLibri2Mix/libri2mix_test-clean.csv in /kaggle/working/Libri2Mix/wav8k/max/test
100%|███████████████████████████████████████| 3000/3000 [00:44<00:00, 67.92it/s]


**Install SpeechBrain**

In [1]:
!pip install speechbrain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 15.1 MB/s eta 0:00:0000:01


In [5]:
!pip install torchmetrics

In [9]:
import os
import torch
import torchaudio
from torch.utils.data import Dataset

from speechbrain.inference.separation import SepformerSeparation as separator
from torch.utils.data import DataLoader
from torchmetrics.functional.audio import scale_invariant_signal_noise_ratio
from torchmetrics.functional.audio import signal_distortion_ratio
import torchaudio.transforms as T

**Dataset**

In [10]:
class Libri2MixDataset(Dataset):
    def __init__(self, root_dir, mix_type='mix_clean', transform=None):
        self.root_dir = root_dir
        self.mix_type = mix_type
        self.transform = transform
        self.mix_files = sorted(os.listdir(os.path.join(root_dir, f"wav8k/min/test/{mix_type}")))
        self.s1_files = sorted(os.listdir(os.path.join(root_dir, "wav8k/min/test/s1")))
        self.s2_files = sorted(os.listdir(os.path.join(root_dir, "wav8k/min/test/s2")))

    def __len__(self):
        return len(self.mix_files)

    def __getitem__(self, idx):
        mix_path = os.path.join(self.root_dir, f"wav8k/min/test/{self.mix_type}", self.mix_files[idx])
        s1_path = os.path.join(self.root_dir, "wav8k/min/test/s1", self.s1_files[idx])
        s2_path = os.path.join(self.root_dir, "wav8k/min/test/s2", self.s2_files[idx])

        mix_waveform, _ = torchaudio.load(mix_path)
        s1_waveform, _ = torchaudio.load(s1_path)
        s2_waveform, _ = torchaudio.load(s2_path)

        sample = {
            "mix": mix_waveform,
            "s1": s1_waveform,
            "s2": s2_waveform
        }

        if self.transform:
            sample = self.transform(sample)

        return sample


In [11]:
# Create dataset for mix_clean type
dataset_mix_clean = Libri2MixDataset(root_dir='/kaggle/working/Libri2Mix', mix_type='mix_clean')

# Create dataset for mix_both type
dataset_mix_both = Libri2MixDataset(root_dir='/kaggle/working/Libri2Mix', mix_type='mix_both')

# Create dataset for mix_single type
dataset_mix_single = Libri2MixDataset(root_dir='/kaggle/working/Libri2Mix', mix_type='mix_single')


In [12]:
from torch.utils.data import random_split


# Define the sizes for training and testing datasets
train_size = int(0.7 * len(dataset_mix_clean))
test_size = len(dataset_mix_clean) - train_size

# Split the mix_clean dataset into training and testing datasets
train_dataset_mix_clean, test_dataset_mix_clean = random_split(dataset_mix_clean, [train_size, test_size])

# Repeat the same process for mix_both dataset
train_dataset_mix_both, test_dataset_mix_both = random_split(dataset_mix_both, [train_size, test_size])

# Repeat the same process for mix_single dataset
train_dataset_mix_single, test_dataset_mix_single = random_split(dataset_mix_single, [train_size, test_size])


In [14]:
# For train datasets
print("Train Dataset: mix_clean")
print("Number of samples:", len(train_dataset_mix_clean))
# print("Example sample:")
#print(train_dataset_mix_clean[0])  # Print first sample

print("\nTrain Dataset: mix_both")
print("Number of samples:", len(train_dataset_mix_both))
# print("Example sample:")
#print(train_dataset_mix_both[0])  # Print first sample

print("\nTrain Dataset: mix_single")
print("Number of samples:", len(train_dataset_mix_single))
# print("Example sample:")
#print(train_dataset_mix_single[0])  # Print first sample

# For test datasets
print("\nTest Dataset: mix_clean")
print("Number of samples:", len(test_dataset_mix_clean))
# print("Example sample:")
#print(test_dataset_mix_clean[0])  # Print first sample

print("\nTest Dataset: mix_both")
print("Number of samples:", len(test_dataset_mix_both))
# print("Example sample:")
#print(test_dataset_mix_both[0])  # Print first sample

print("\nTest Dataset: mix_single")
print("Number of samples:", len(test_dataset_mix_single))
print("Example sample:")
print(test_dataset_mix_single[0])  # Print first sample


Train Dataset: mix_clean
Number of samples: 2100

Train Dataset: mix_both
Number of samples: 2100

Train Dataset: mix_single
Number of samples: 2100

Test Dataset: mix_clean
Number of samples: 900

Test Dataset: mix_both
Number of samples: 900

Test Dataset: mix_single
Number of samples: 900
Example sample:
{'mix': tensor([[-0.0061, -0.0069, -0.0145,  ..., -0.0139,  0.0208,  0.0755]]), 's1': tensor([[-3.0518e-05, -3.0518e-05,  0.0000e+00,  ..., -4.2419e-03,
          2.4353e-02,  6.8237e-02]]), 's2': tensor([[-5.4932e-04, -6.1035e-05, -5.7983e-04,  ..., -3.3569e-04,
         -1.0376e-03,  3.6621e-04]])}


In [40]:
from torch.utils.data import DataLoader

# For train datasets
loader_train_mix_clean = DataLoader(train_dataset_mix_clean, batch_size=1, shuffle=False)
loader_train_mix_both = DataLoader(train_dataset_mix_both, batch_size=1, shuffle=False)
loader_train_mix_single = DataLoader(train_dataset_mix_single, batch_size=1, shuffle=False)

# For test datasets
loader_test_mix_clean = DataLoader(test_dataset_mix_clean, batch_size=1, shuffle=False)
loader_test_mix_both = DataLoader(test_dataset_mix_both, batch_size=1, shuffle=False)
loader_test_mix_single = DataLoader(test_dataset_mix_single, batch_size=1, shuffle=False)

**Metrices**

In [25]:
import torchaudio

def compute_sisnr(estimated_sources, sources):
  """
  Calculates Scale-Invariant Signal-to-Noise Ratio (SISNR)

  Args:
      estimated_sources: Separated source waveforms (torch.Tensor)
      sources: Ground truth source waveforms (torch.Tensor)

  Returns:
      SISNR for each source (torch.Tensor)
  """
  eps = 1e-7  # Adjusted epsilon value
  mixture = estimated_sources.sum(dim=0)  # Sum estimated sources to get mixture
  # Check for zero norms before division and log
  norm_mixture = torch.norm(mixture, dim=1)
  mask = norm_mixture > eps  # Create mask to avoid division by zero
  sisnr = 20 * torch.log10(torch.where(mask, torch.norm(sources, dim=1) / norm_mixture, torch.zeros_like(norm_mixture)))
  return sisnr

def compute_sdri(estimated_sources, sources, mixture):
  """
  Calculates Signal-to-Distortion Ratio improvement (SDRi)

  Args:
      estimated_sources: Separated source waveforms (torch.Tensor)
      sources: Ground truth source waveforms (torch.Tensor)
      mixture: Mixed audio signal (torch.Tensor)

  Returns:
      SDRi for each source (torch.Tensor)
  """
  eps = 1e-7  # Adjusted epsilon value
  min_norm = 1e-5  # Minimum threshold for norm
  # Calculate SDR with thresholding for norms
  est_sdr = 20 * torch.log10(torch.clamp(torch.norm(sources, dim=1), min=min_norm) / (torch.norm(estimated_sources - sources, dim=1) + eps))
  mix_sdr = 20 * torch.log10(torch.clamp(torch.norm(sources, dim=1), min=min_norm) / (torch.norm(mixture, dim=1) + eps))
  # SDRi calculation remains the same
  sdri = est_sdr - mix_sdr
  return sdri

**Model**

In [ ]:
# Load the model
model = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir='pretrained_models/sepformer-whamr')


In [26]:
# Function to evaluate the model on a given DataLoader
def evaluate_model(loader):
    sdr_scores = []
    sisnr_scores = []
    with torch.no_grad():
        for i, sample in enumerate(loader):
            mix = sample['mix'].squeeze(0)  # Remove batch dimension
            sources = torch.stack([sample['s1'].squeeze(0), sample['s2'].squeeze(0)])  # Stack source waveforms along new dimension


            # Perform separation
            estimated_sources = model.separate_batch(mix) 
            
            

            # Compute SDRi and SISNRi
            sdr = compute_sdri(estimated_sources.permute(0, 2, 1), sources.permute(1, 0, 2), mix)
            sisnr = compute_sisnr(estimated_sources.permute(0, 2, 1), sources.permute(1, 2, 0))
            
#             print("sdr", sdr)
#             print("sisnr", sisnr)

            # Store SDR and SISNR for each source
            sdr_scores.append(sdr.mean().item())  # Compute mean SDR over all sources
            sisnr_scores.append(sisnr.mean().item())  # Compute mean SISNR over all sources

    mean_sdr = sum(sdr_scores) / len(sdr_scores)
    mean_sisnr = sum(-sisnr_scores) / len(sisnr_scores)
    return mean_sisnr, mean_sdr

In [35]:
# Evaluate each dataset
mean_sisnr_mix_clean, mean_sdri_mix_clean = evaluate_model(loader_test_mix_clean)

print(f"Mean SISNRi: {mean_sisnr_mix_clean:.2f} dB")
print(f"Mean SDRi: {mean_sdri_mix_clean:.2f} dB")

In [ ]:
mean_sisnr_mix_both, mean_sdri_mix_both = evaluate_model(loader_test_mix_both)

print(f"Mean SISNRi: {mean_sisnr_mix_both:.2f} dB")
print(f"Mean SDRi: {mean_sdri_mix_both:.2f} dB")


In [ ]:
mean_sisnr_mix_single, mean_sdri_mix_single = evaluate_model(loader_test_mix_single)

print(f"Mean SISNRi: {mean_sisnr_mix_single:.2f} dB")
print(f"Mean SDRi: {mean_sdri_mix_single:.2f} dB")

**Finetuning**

In [22]:
import torch

def sisnr_loss(estimated_sources, reference_sources):
    # Center the estimated sources to have zero mean
    estimated_sources = estimated_sources - torch.mean(estimated_sources, dim=-1, keepdim=True)

    # Compute the energy of the reference sources
    reference_energy = torch.sum(reference_sources ** 2, dim=-1, keepdim=True)

    # Compute the dot product between estimated sources and reference sources
    dot_product = torch.sum(estimated_sources * reference_sources, dim=-1, keepdim=True)

    # Compute the scale factor
    scale_factor = dot_product / reference_energy

    # Compute the noise signal
    noise = estimated_sources - scale_factor * reference_sources

    # Compute SI-SNR
    sisnr = 10 * torch.log10(torch.sum(reference_sources ** 2, dim=-1) / torch.sum(noise ** 2, dim=-1))

    # Return the negative SI-SNR (minimize SI-SNR loss)
    return -torch.mean(sisnr)


In [4]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import MSELoss
import matplotlib.pyplot as plt

from speechbrain.inference.separation import SepformerSeparation as separator
import torchaudio

model = separator.from_hparams(source="speechbrain/sepformer-wsj02mix", savedir='pretrained_models/sepformer-wsj02mix')

# Define loss function and optimizer
# criterion = MSELoss()
optimizer = Adam(model.parameters(), lr=0.01)

# Define training loop
def train(loader):
    model.train()
    total_loss = 0.0
    for i, sample in enumerate(loader):
        mix = sample['mix'].squeeze(0)  # Remove batch dimension
        sources = torch.stack([sample['s1'].squeeze(0), sample['s2'].squeeze(0)]).requires_grad_()   # Stack source waveforms along new dimension

        # Forward pass
        estimated_sources = model.separate_batch(mix)

        # Compute SI-SNR loss
        loss = sisnr_loss(estimated_sources.permute(0, 2, 1), sources.permute(1, 0, 2))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)



# Function to evaluate the model on a given DataLoader
def evaluate(loader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for i, sample in enumerate(loader):
            mix = sample['mix'].squeeze(0)  # Remove batch dimension
            sources = torch.stack([sample['s1'].squeeze(0), sample['s2'].squeeze(0)])  # Stack source waveforms along new dimension

            # Perform separation
            estimated_sources = model.separate_batch(mix) 

            # Compute loss
            loss = criterion(estimated_sources, sources)

            total_loss += loss.item()

    return total_loss / len(loader)


hyperparams.yaml:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

masknet.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

encoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

decoder.ckpt:   0%|          | 0.00/17.2k [00:00<?, ?B/s]

In [43]:
# Train the model
num_epochs = 10
train_losses = []

for epoch in range(num_epochs):
    train_loss = train(loader_train_mix_clean)  # Use the appropriate train loader
    train_losses.append(train_loss)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss}")

# Print the training losses
print("Training losses:", train_losses)


Epoch 1/10, Train Loss: 15.696583112080893
Epoch 2/10, Train Loss: 15.696583112080893
Epoch 3/10, Train Loss: 15.696583112080893
Epoch 4/10, Train Loss: 15.696583112080893
Epoch 5/10, Train Loss: 15.696583112080893
Epoch 6/10, Train Loss: 15.696583112080893
Epoch 7/10, Train Loss: 15.696583112080893
Epoch 8/10, Train Loss: 15.696583112080893
Epoch 9/10, Train Loss: 15.696583112080893
Epoch 10/10, Train Loss: 15.696583112080893
Training losses: [15.696583112080893, 15.696583112080893, 15.696583112080893, 15.696583112080893, 15.696583112080893, 15.696583112080893, 15.696583112080893, 15.696583112080893, 15.696583112080893, 15.696583112080893]


In [30]:
# Evaluate each dataset
mean_sisnr_mix_clean, mean_sdri_mix_clean = evaluate_model(loader_test_mix_clean)

print(f"Mean SISNRi: {mean_sisnr_mix_clean:.2f} dB")
print(f"Mean SDRi: {mean_sdri_mix_clean:.2f} dB")

Mean SISNRi: -27.94 dB
Mean SDRi: 27.98 dB


In [31]:
mean_sisnr_mix_both, mean_sdri_mix_both = evaluate_model(loader_test_mix_both)

print(f"Mean SISNRi: {mean_sisnr_mix_both:.2f} dB")
print(f"Mean SDRi: {mean_sdri_mix_both:.2f} dB")


Mean SISNRi: -28.97 dB
Mean SDRi: 23.80 dB


In [32]:
mean_sisnr_mix_single, mean_sdri_mix_single = evaluate_model(loader_test_mix_single)

print(f"Mean SISNRi: {mean_sisnr_mix_single:.2f} dB")
print(f"Mean SDRi: {mean_sdri_mix_single:.2f} dB")

Mean SISNRi: -24.04 dB
Mean SDRi: 25.35 dB


**Create Gradio link**

In [1]:
# !pip install gradio
# import gradio
# import torch

In [15]:
pip install -U gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 73.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 91.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.1 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=723d3e62dbca68b624bdaf1d889dfd2ea6f5b193e37838e40b9b47ac47f7039a
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.12.4
    Uninstalling tomlkit-0.12.4:
      Successfully uninstalled tomlkit-0.12.4
Note: you may nee

In [28]:
!git clone https://huggingface.co/spaces/iiserkbikram/sepformer_finetuned

Cloning into 'sepformer_finetuned'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (4/4), 1.29 KiB | 1.29 MiB/s, done.


In [29]:
# iface = gr.Interface(fn=greet, inputs="text", outputs="text")
# iface.launch()

In [35]:
# %%writefile app.py

import os
import numpy as np
import gradio as gr

# Define a function to use the trained model for inference
def separate_audio(input_audio):
    # Assuming 'model' is your trained model
    # Perform inference on the input audio
    separated_sources = model.separate_batch(input_audio)

    # Return the separated audio sources
    return separated_sources

input_audio = gr.Audio(
    sources=["microphone"],
    waveform_options=gr.WaveformOptions(
        waveform_color="#01C6FF",
        waveform_progress_color="#0066B4",
        skip_length=2,
        show_controls=False,
    ),
)
demo = gr.Interface(
    fn=separate_audio,
    inputs=input_audio,
    outputs="audio"
)

if __name__ == "__main__":
    demo.launch()


Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://d7ff9994e464badbe3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [31]:
!git config --global user.email "iiserkbikram@gmail.com"
!git config --global user.name "bikrammajhi"

In [ ]:
!git add app.py
!git commit -m "sepformer finetuned"
!git push